
Part B

Pick your favorite between 'query_original' and 'query_dt_c' and let's add some more functionality.

Queries used daily are stored in a csv file in the drive(queries.csv).

Make your class able to instantiate from this csv and create as many instances as lines existing in the csv.
Each line in the csv has two columns 'Query' and 'Number' as supposed.
Csv file path argument shall be optional. If a path is provided it will be used or a default path should be used.
Print the 'query' string of all instances created from the csv. (not instances as whole)

In [50]:
import pandas as pd
from collections.abc import Iterable


class Query:
    def __init__(self, query_string: str, no_of_query: int = 0):
        self.query_string = query_string
        self.no_of_query = no_of_query

    @property
    def no_of_query(self):
        return self._no_of_query

    @no_of_query.setter
    def no_of_query(self, n):
        if n < 0:
            raise ValueError(
                f"no_of_query must be zero or greater not {n}"
            )
        self._no_of_query = n

    def __str__(self):
        return f"{self.__class__.__name__} object qstring={self.query_string}, no_of_query={self.no_of_query}"


class QueryOriginal:
    def __init__(self, queries: str | Iterable[Query] = "queries.csv"):
        if isinstance(queries, str):
            self._load_queries_from_csv(queries)
        elif isinstance(queries, Iterable) and all(
            map(lambda x: isinstance(x, Query), queries)
        ):
            self.query_list = [query for query in queries]
        else:
            raise ValueError(
                "Can only pass a str or an Iterable of Query objects"
            )
    def __iter__(self):
        return iter(self.query_list)

    def _load_queries_from_csv(self, csvpath: str):
        df = pd.read_csv(csvpath)
        self.query_list = [Query(t[1], t[2]) for t in df.itertuples()]

    def __str__(self):
        class_name = self.__class__.__name__
        queries = "\n".join(map(str, self.query_list))
        return f"{class_name}: \n{queries}"

In [51]:
query_collection = QueryOriginal()

In [52]:
print(query_collection)

QueryOriginal: 
Query object qstring=csvtest1, no_of_query=11
Query object qstring=csvtest2, no_of_query=22
Query object qstring=csvtest3, no_of_query=33
Query object qstring=csvtest0, no_of_query=0


In [53]:
for query in query_collection:
    print(query)

Query object qstring=csvtest1, no_of_query=11
Query object qstring=csvtest2, no_of_query=22
Query object qstring=csvtest3, no_of_query=33
Query object qstring=csvtest0, no_of_query=0


create a guessing game to be initialized as a class method. You can for example pick one random integer from 0-10 and have the user guess it in three tries.
Hint: Method for step 3 will not access or use any class attribute. So what type of method will be used?

In [1]:
import random


class GuessingGame:
    @staticmethod
    def start_game():
        correct = random.randint(0, 10)
        print("Start  of game, guess a number from 0 to 10")
        for i in range(3):
            answer = None
            while answer is None:
                answer = input(f"you got {3 - i} tries: ")
                try:
                    answer = int(answer)
                except:
                    answer = None
            if answer == correct:
                print("Congrats! You won!")
                break
            else:
                print("wrong answer")
        else:
            print("Game Over")
        play_again = None
        while play_again not in ("y", "n"):
            play_again = input(
                "Would you like to play again? ('y', 'n'): "
            )
        if play_again == "y":
            GuessingGame.start_game()

In [ ]:
GuessingGame.start_game()

Part C  

Convert your web scraping code you developed previously into object oriented (use code from flexcar or atp rankings).

## Refactor ATP Scraping Exercise

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging
import time
import random


logger = logging.getLogger(__name__)
logger.addHandler(logging.FileHandler("log.txt"))
logger.setLevel(logging.INFO)


class Player:
    def __init__(self, name: str, country: str, points: int):
        self.name = name
        self.country = country
        self.points = points


class Week:
    def __init__(self, url: str):
        self.ranks = range(1, 101)
        self._players = None


class AtpDb:
    def __init__(self, soup: BeautifulSoup, weeks: list[Week] = None):
        self._weeks = [] if weeks is None else weeks
        self._soup = soup
        self.response_list = []
        self._headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
        }

    def add_week(self, week):
        self._weeks.append(week)

    def add_weeks(self, weeks: list[Week]):
        self._weeks.extend(weeks)

    @property
    def weeks(self):
        return self._weeks

    @property
    def soup(self):
        return self._soup

    def get_all_urls(self) -> list[str]:
        """Return a list with all weekly urls from the main soup object"""
        select_tag = self.soup.find("select", id="dateWeek-filter")
        date_tags = select_tag.find_all("option")
        dates = [
            (
                date_tag["value"]
                if not "Current" in date_tag["value"]
                else "Current+Date"
            )
            for date_tag in date_tags
        ]
        base_url = (
            "https://www.atptour.com/en/rankings/singles?dateWeek="
        )
        return sorted([f"{base_url}{date}" for date in dates])

    def get_responses(self):
        urls = self.get_all_urls()
        urls_length = len(urls)

        for i, url in enumerate(urls):
            logger.info(
                f"processing request {i + 1} out of {urls_length}"
            )
            time.sleep(random.uniform(0.3, 0.4))
            response = requests.get(url, headers=self._headers)
            if response.status_code != 200:
                response_list = []
                raise ConnectionError(
                    f"failed to fetch data from {url}"
                )
            self.response_list.append(response)

#### Original Exercise

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd


def get_all_urls(soup: BeautifulSoup) -> list[str]:
    """Return a list with all weekly urls from the main soup object"""
    select_tag = soup.find("select", id="dateWeek-filter")
    date_tags = select_tag.find_all("option")
    dates = [date_tag["value"] if not "Current" in date_tag["value"] else "Current+Date" for date_tag in date_tags]
    base_url = "https://www.atptour.com/en/rankings/singles?dateWeek="
    return sorted([f"{base_url}{date}" for date in dates])


def get_active_week(soup: BeautifulSoup) -> str:
    """
    returns the active week, directly through scraping and not through the url
    unlike the get_all_urls it will extract the text contents instead of 
    the value attribute, otherwise we would get date 'Current-Date' for our
    most recent date.
    """
    
    select_tag = soup.find("select", id="dateWeek-filter")
    return select_tag.find("option", selected=True).text


def get_player_names(soup: BeautifulSoup) -> list[str]:
    """
    returns a list of the top-100 players names ordered by rank
    """
    li_tags = soup.find_all("li", class_="name center")
    return [li.find("span").text for li in li_tags]


def get_countries(soup: BeautifulSoup) -> list[str]:
    """
    returns a list of the top-100 players countries ordered by rank
    """
    svg_tags = soup.find_all("svg", class_="atp-flag")[:100]
    use_tags = [svg_tag.find("use") for svg_tag in svg_tags]
    links = [use_tag["href"] for use_tag in use_tags]
    countries = [_extract_flag_abbr(link) for link in links]
    return _convert_flag_abbr(soup, countries)


def _extract_flag_abbr(string: str) -> str:
    """
    extract the flag abbreviation out of a link to the flag png
    """
    return re.search(r"(?<=#flag-)[A-Za-z]{3}$", string).group(0)


def _convert_flag_abbr(
    soup: BeautifulSoup, countries: list[str]
) -> str:
    """
    helper function to convert a flag/country abbreviation to the 
    actual country name. The relationship between the can be find inside
    the source code and I did not have rely on external sources
    """
    select_region_filter = soup.find("select", id="region-filter")
    region_option_tags = select_region_filter.find_all("option")
    countries_tuple = [
        (region_option_tag["value"], region_option_tag.text)
        for region_option_tag in region_option_tags
    ]
    country_dict = {k.lower(): v for k, v in countries_tuple}
    return [
        country_dict.get(country_abbr, country_abbr)
        for country_abbr in countries
    ]


def get_points(soup: BeautifulSoup) -> list[str]:
    """
    after some debugging a edge case failures, I adjusted and tested
    the points extraction to the following code.
    I essentially had to find the tag before the one I was looking for, 
    and the seek the sibling.
    The reason for picking the slice is that I sometimes get 101 results 
    instead of 100 and in that case the first one does not lead to any points.
    Taking the slice of the last 100 seems safe.
    """
    points_tds = soup.find_all("td", class_="age small-cell")
    return [
        points_td.find_next_sibling("td").find("a").text.strip()
        for points_td in points_tds[-100:]
    ]

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
}

url = "https://www.atptour.com/en/rankings/singles"
response = requests.get(url, headers=headers)
response.status_code
# Check status code
if response.status_code != 200:
    print(response.status_code, response.reason)

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
import logging

logger = logging.getLogger(__name__)
logger.addHandler(logging.FileHandler("log.txt"))
logger.setLevel(logging.INFO)

In [ ]:
# gather all responses

import time
import random


urls = get_all_urls(soup)
urls_length = len(urls)

response_list = []

for i, url in enumerate(urls):
    logger.info(f"processing request {i + 1} out of {urls_length}")
    time.sleep(random.uniform(0.3, 0.4))
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        response_list = []
        raise ConnectionError(f'failed to fetch data from {url}')
    response_list.append(response)

In [ ]:
# save html to files
for i, response in enumerate(response_list):
    if response and response.status_code == 200:
        with open(f"pages/page_{i}.html", "wb") as f:
            f.write(response.content)

In [ ]:
#load files to a list of html text
import glob

content_list = []
file_paths = sorted(glob.glob("pages/page_*.html"))

for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        content_list.append(f.read())

In [ ]:
# convert all elements of the html list to a list of soup objects (parsed)
# extract data and load them into a list of dictionaries
content_length = len(content_list)
results_list = []

for i, content in enumerate(content_list):
    logger.info(f"processing response {i + 1} out of {content_length}")
    soup = BeautifulSoup(content, "html.parser")
    week = get_active_week(soup)
    ranks = list(range(1, 101))
    player_names = get_player_names(soup)
    countries = get_countries(soup)
    points_all = get_points(soup)
    sizes = map(len, [ranks, player_names, countries, points_all])
    # if not len(set(sizes)) == 1 or not all(v > 0 for v in sizes):
    if not all(v > 0 for v in sizes):
        logger.info(
            f"week {week} error\nranks: {ranks}\nplayer_names: {player_names}\ncountries: {countries} \n points_all: {points_all}"
        )

    results_list.extend(
        [
            {
                "week": week,
                "rank": rank,
                "player_name": player_name,
                "country": country,
                "points": points,
            }
            for rank, player_name, country, points in zip(
                ranks, player_names, countries, points_all
            )
        ]
    )